In [14]:
import numpy as np
import pandas as pd
import pprint
import datetime as DT

In [15]:
today = DT.date.today()
print("Today's date:", today)

Today's date: 2021-05-17


In [16]:
pp = pprint.PrettyPrinter(depth=6)

In [17]:
#User inputs
nb_days = 2
nb_pers = 2.25

In [18]:
# import all meal menus
menus = ['breakfast','lunch','dinner','snack']
ls = []
my_len = []
for item in menus:
    # import all menus
    menu_df = pd.read_excel('meniu.xlsx', sheet_name=item)
    menu_df['Index'] = pd.Series(menu_df['Index']).fillna(method='ffill')
    menu_df[item] = pd.Series(menu_df[item]).fillna(method='ffill')
    my_len.append(int(menu_df['Index'].max()))
    # Create Multi-index
    menu_df.set_index(['Index', item,'Ingrediente'], inplace=True)
    menu_df.sort_index(inplace=True)
    ls.append(menu_df)

In [19]:
# generate random menu
import random as rd
def rand_nb_func(my_len,nb_days):
    rand_nb = rd.sample(range(1,my_len), nb_days)
    return rand_nb

In [20]:
# call random function for every meal
ls_rand=[]
nb_meals = range(len(menus))
for item in nb_meals:
    ls_rand.append(rand_nb_func(my_len[item],nb_days))
#print(ls_rand)

In [21]:
# select menu for nb_days needed
ls_myDataFrame=[]
my_date=[]
for item in nb_meals:
    myDataFrame = pd.DataFrame()
    for i in range(0,nb_days):
        #print(ls_rand[item][i])
        myDataFrame = myDataFrame.append(ls[item].loc[ls_rand[item][i]])
        
    ls_myDataFrame.append(myDataFrame)
    # print the menu for n days

In [22]:
ls_myDataFrame[0]

Quantity [g]  Quantity [buc]
breakfast                    Ingrediente                                    
Mix Ou fiert avocado telemea Avocado                     100               0
                             Felie paine de in             0               1
                             Oua                           0               1
                             Telemea                      50               0
Tartine somon castravete     Almette                      50               0
                             Avocado                     100               0
                             Castraveti                  100               0
                             Felie paine de in             0               1
                             Somon fume                   50               0

In [23]:
ls_myDataFrame[1]

Quantity [g]  Quantity [buc]
lunch                    Ingrediente                                       
Salata de ton            Avocado                        100             0.0
                         Castraveti                     150             0.0
                         Ceapa                            0             1.0
                         Lingura ulei masline            15             0.0
                         Masline                         30             0.0
                         Salata verde                   100             0.0
                         Ton in suc propriu             130             0.0
Ficatei cu brocoli fiert Brocoli                        200             0.0
                         Ficatei de pui                 200             0.0
                         Lime                             0             0.5
                         Lingura ulei masline            15             0.0
                         Unt                             10             0.0
                         Usturoi                         10             0.0

In [24]:
ls_myDataFrame[2]

Quantity [g]  \
dinner                         Ingrediente                          
Grill cu halloumi si salata    Baby spanac                    100   
                               Halloumi la grill              150   
                               Lingura ulei masline            15   
                               Rosii cherry                   100   
Dovlecei sotati cu sos tzatiky Castraveti                     100   
                               Dovlecei                       250   
                               Iaurt grecesc 2%                 0   
                               Lime                             0   
                               Lingura ulei masline            15   
                               Unt                             15   
                               Usturoi                         10   

                                                     Quantity [buc]  
dinner                         Ingrediente                           
Grill cu halloumi si salata    Baby spanac                      0.0  
                               Halloumi la grill                0.0  
                               Lingura ulei masline             0.0  
                               Rosii cherry                     0.0  
Dovlecei sotati cu sos tzatiky Castraveti                       0.0  
                               Dovlecei                         0.0  
                               Iaurt grecesc 2%                 1.0  
                               Lime                             0.5  
                               Lingura ulei masline             0.0  
                               Unt                              0.0  
                               Usturoi                          0.0

In [25]:
# export list of ingredients by pieces
my_df_piece = []
for item in nb_meals:
    my_df_piece.append(ls_myDataFrame[item].groupby('Ingrediente')['Quantity [buc]'].sum().reset_index())
    my_df_piece[item] = my_df_piece[item][my_df_piece[item]['Quantity [buc]']!= 0]
    my_df_piece[item]['Family Quantity [buc]'] = my_df_piece[item]['Quantity [buc]']* nb_pers

df_merged_piece_1 = pd.merge(my_df_piece[0], my_df_piece[3], how='outer')
df_merged_piece_2 = pd.merge(my_df_piece[1], my_df_piece[2], how='outer')
df_merged_piece = pd.merge(df_merged_piece_1, df_merged_piece_2, how='outer')

df_merged_piece.groupby('Ingrediente').sum()

E:\Engineering_software\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


,Quantity [buc],Family Quantity [buc]
Ingrediente,,
Ceapa,1.0,2.250
Felie paine de in,2.0,4.500
Iaurt grecesc 2%,1.0,2.250
Lime,0.5,1.125
Oua,1.0,2.250
Rodie,1.0,2.250


In [26]:
# export list of ingredients by grams
my_df_quantity = []
for item in nb_meals:
    my_df_quantity.append(ls_myDataFrame[item].groupby('Ingrediente')['Quantity [g]'].sum().reset_index())
    my_df_quantity[item] = my_df_quantity[item][my_df_quantity[item]['Quantity [g]']!= 0]
    my_df_quantity[item]['Family Quantity [g]'] = my_df_quantity[item]['Quantity [g]']* nb_pers

df_merged_quantity_1 = pd.merge(my_df_quantity[0], my_df_quantity[3], how='outer')
df_merged_quantity_2 = pd.merge(my_df_quantity[1], my_df_quantity[2], how='outer')
df_merged_quantity = pd.merge(df_merged_quantity_1, df_merged_quantity_2, how='right')  
    
df_merged_quantity.groupby('Ingrediente').sum()


,Quantity [g],Family Quantity [g]
Ingrediente,,
Avocado,100,225.00
Baby spanac,100,225.00
Brocoli,200,450.00
Castraveti,250,562.50
Dovlecei,250,562.50
Ficatei de pui,200,450.00
Halloumi la grill,150,337.50
Lingura ulei masline,30,67.50
Masline,30,67.50
